
# Chatbot IA avec Rag

Ce notebook implémente un chatbot IA avancé en utilisant **LangChain** et **LangGraph**.  
Il intègre :
- **RAG (Retrieval-Augmented Generation) avec affichage des sources**
- **Streaming des réponses pour une meilleure expérience utilisateur**
- **Mémoire conversationnelle pour un suivi des interactions**


In [3]:

!pip install --upgrade --quiet langchain langchain-community langchainhub beautifulsoup4 langchain-openai beautifulsoup4 chromadb
!pip install --quiet langgraph
!pip install --upgrade langchain-mistralai



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: pip install --upgrade pip


##  Importation des bibliothèques

In [10]:
# Importation des bibliothèques

import os
import json
import getpass

from langchain_mistralai.chat_models import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver

## Connexion à l'Api 

In [11]:
# Configuration API Mistral AI

MISTRAL_API_KEY = getpass.getpass("Entrez votre clé API Mistral: ")  
MODEL_NAME = "mistral-small-latest" #test du modèle small pour combiner performance et prix accessible


In [12]:
# Initialisation du modèle / activation du streaming
llm = ChatMistralAI(
    mistral_api_key=MISTRAL_API_KEY,
    model=MODEL_NAME,
    streaming=True  
)

##  Définition du Prompt

In [13]:
# Définition du Prompt

prompt_template = ChatPromptTemplate.from_template(
    """
    Tu es un assistant d'orientation professionnelle. Ton rôle est d'aider l'utilisateur à faire des choix de carrière en fonction de son parcours, sa personnalité, ses compétences et ses préférences. Adapte toi à l'utilisateur. Soit complet mais concis.

    CONTEXTE:
    ---------------------
    {context}
    ---------------------

    Question de l'utilisateur: {input}

    Fournis une réponse claire, bien structurée et motivante.
    """
)


##  Définition du Chatbot avec mémoire, RAG

In [14]:
# Mise en place de la mémoire conversationnelle

memory = MemorySaver()
workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    """Appelle le modèle avec historique de conversation"""
    response = llm.invoke(state["messages"])
    return {"messages": response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
chatbot = workflow.compile(checkpointer=memory)


In [15]:
# Fonction pour générer une réponse

def generate_response(user_query: str):
    """ Génère une réponse sans utiliser ChromaDB."""

    # Pas de récupération de contexte depuis une base de données vectorielle
    context = "Pas de contexte"

    # Construction du prompt
    formatted_prompt = prompt_template.format(input=user_query, context=context)

    # Appel au modèle Mistral
    response = llm.invoke(formatted_prompt)

    return {
        "response": response,
        "sources": ["Aucune source (VDB désactivée)"]
    }

## Test du Chatbot

In [8]:
# Test du chatbot avec une question utilisateur

user_input = "Je suis étudiant en informatique et j'hésite entre devenir développeur ou data scientist. Que me recommandes-tu ?"
result = generate_response(user_input)

print("\n Réponse du chatbot :")
print(result["response"])

print("\n Sources utilisées :")
for src in result["sources"]:
    print(f" - {src}")




 Réponse du chatbot :
content="Bien sûr, je serais ravi de t'aider à faire ce choix important !\n\n### Développeur\n\n**Description du rôle :**\n- **Responsabilités :** Conception, développement et maintenance de logiciels, applications web, et systèmes informatiques.\n- **Compétences clés :** Programmation (langages comme Python, Java, JavaScript, etc.), développement de logiciels, résolution de problèmes, gestion de projets.\n- **Environnement de travail :** Souvent dans des bureaux, avec des équipes de développement, mais le télétravail est également courant.\n- **Perspectives de carrière :** Les développeurs peuvent évoluer vers des rôles de lead developer, architecte logiciel, ou même de chef de projet.\n\n**Avantages :**\n- **Créativité :** Tu peux travailler sur des projets variés et innovants.\n- **Demande élevée :** Il y a une forte demande pour les développeurs dans de nombreux secteurs.\n- **Autonomie :** Possibilité de travailler en freelance ou dans des startups dynamique

In [16]:
# Test du chatbot avec une question utilisateur 2

user_input = "Je suis au lycée, je ne sais pas quel métier je veux faire mais je sais que je suis ENTP sur ma personnalité MBTI, que me conseilles tu ?"
result = generate_response(user_input)

print("\n Réponse du chatbot :")
print(result["response"])

print("\n Sources utilisées :")
for src in result["sources"]:
    print(f" - {src}")



 Réponse du chatbot :
content="Bien sûr, je comprends que tu es au lycée et que tu cherches à orienter ta carrière. Étant donné que tu es ENTP selon le test de personnalité MBTI, voici quelques conseils et suggestions de métiers qui pourraient correspondre à ton profil:\n\n### Caractéristiques des ENTP:\n1. **Créatifs et innovants**: Les ENTP aiment résoudre des problèmes complexes et inventer de nouvelles idées.\n2. **Analytiques et logiques**: Ils ont une forte capacité à analyser des situations et à prendre des décisions basées sur des données.\n3. **Communicateurs efficaces**: Les ENTP sont bons pour expliquer des concepts complexes de manière claire et convaincante.\n4. **Aventuriers et curieux**: Ils aiment explorer de nouveaux domaines et ne se contentent pas de la routine.\n\n### Suggestions de métiers:\n1. **Ingénieur en informatique**: Ce domaine te permettra d'utiliser tes compétences analytiques et ta créativité pour développer des solutions technologiques.\n2. **Consultan